# Causal Classifier - Model Training

This notebook is used to train a causal classifier (e.g., LSTM) as part of the modular FUDGE project. It handles data loading, model initialization, and the complete training loop. You can run all cells to train a model from scratch. For a complete overview of the project architecture and how this script is used, please see the [full project on the GitHub repo](https://github.com/latoohey/modular-fudge). The testing script to create a classifier for guiding generation is also available as a [Colab notebook](https://colab.research.google.com/drive/1RlbP7iAJY4Ajga93tRy0KjxPwe558Yb9?usp=sharing).

## Imports

In [ ]:
!pip install transformers scikit-learn torch

In [ ]:
# The Mamba related packages are slow to build
# Leave these installs commented unless you intend to train a Mamba model
#
import os
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define the directory on your Drive to store the wheels
#    We use a specific folder name to keep it organized.
wheel_dir = '/content/drive/MyDrive/colab_wheels/mamba_builds'
os.makedirs(wheel_dir, exist_ok=True)

# 3. Define the package versions you want
packages = [
    "causal-conv1d>=1.4.0",
    "mamba-ssm"
]

# 4. Check if wheels already exist in your Drive
print(f"Checking for existing wheels in {wheel_dir}...")
existing_wheels = [f for f in os.listdir(wheel_dir) if f.endswith('.whl')]

if len(existing_wheels) >= len(packages):
    print("✅ Found pre-built wheels! Installing from Drive...")
    # Install directly from your Drive folder
    !pip install "$wheel_dir"/*.whl
else:
    print("⚠️ No wheels found. Building from source (this will take time once)...")

    # Install build dependencies first
    !pip install packaging ninja

    # Build the wheels and save them directly to your Drive
    # We use --no-deps to avoid building wheels for huge packages like PyTorch
    print(f"Building wheels to {wheel_dir}...")
    !pip wheel {" ".join(packages)} --wheel-dir="$wheel_dir" --no-deps

    # Now install the newly built wheels
    print("Installing newly built wheels...")
    !pip install "$wheel_dir"/*.whl

print("🎉 Done! Mamba and Causal-Conv1d are ready.")
from mamba_ssm import Mamba
import einops

In [ ]:

import os
import random
import time
from datetime import datetime
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader, Dataset as TorchDataset
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from huggingface_hub import login
from transformers import AutoTokenizer
from collections import defaultdict
from types import SimpleNamespace
import sys
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import requests
from pathlib import Path

import os
import torch
import random
from torch.utils.data import DataLoader, Dataset as TorchDataset
from transformers import AutoTokenizer


## Configuration:

Note: Please include your Hugging Face token as a Colab Secret named `HF_TOKEN`

* Set a `SEED` set for reproducability

* `DATA_DIR` is the location where your data is saved it must contain train and test folders and have training data split into text files by class. E.g:
  * data/splits/train/style_0.txt
  * data/splits/train/style_1.txt
  * data/splits/test/style_0.txt
  * data/splits/test/style_1.txt

  The project data can be downloaded from Github by setting the `PROJECT_DATA` parameter to True. This will handle creating the necessary directories.

* `SAVE_DIR` is the folder where your saved model package will be saved. You'll be prompted to log into Google Drive as part of the flow.

* `HF_TOKEN` this can be left blank when running on Colab
* `CKPT` Leave this set to None in Colab

* There are a variety of general training hyperparameters to set including `BATCH_SIZE`, `EPOCHS`, `LR`, `NUM_WORKERS`, and output `PRINT_FREQ`

* The `POS_CAT` and `NEG_CAT` should match the names of the respective .txt files in your train and text folders. They define which category will be the training target.

* `MODEL_TYPE` is the code for the type of model you'll be training. This must match one of the codes in the `get_model()` factory function.

* Models themselves can be any type of binary classifier but they must have:
  1.  A class that inherits from `torch.nn.Module`.
  2.  An `__init__` method with a specific signature.
  3.  An *internal* `forward` method for the model's logic.
  4.  A `get_scores_for_batch` "adapter" method for training.
  5.  A `get_final_scores` "adapter" method for evaluation.

  The full description of model requirements is included [below at the top of the Models section](https://colab.research.google.com/drive/1zVfBB_zIKHpSANBmTcj8KRBFwsPgbd9m#scrollTo=f0717723-a806-42b1-81b4-50de25dd7e15).

* Settings like `LSTM_HIDDEN_DIM` and `LSTM_NUM_LAYERS` are specific to particular model types. You can define more parameters as needed provided that you add them to `traing_args` namespace below which will give access in your model definition.

* Define the `TOKENIZER_NAME` and `PAD_TOKEN` your data will be tokenized into before training.

* `VAL_SIZE` defines the size of the validation set used during training.

In [ ]:
SEED=24601

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

MODEL_TYPE="mamba"

DATA_DIR="data"
SAVE_DIR="trained_models"

HF_TOKEN="XXXXX" # Set automatically from secret if on colab

BATCH_SIZE=64
EPOCHS=100
LR=0.0001
NUM_WORKERS=4
CKPT=None
PRINT_FREQ=100

# --- Tokenizer ---
TOKENIZER_NAME = 'meta-llama/Llama-3.2-3B-Instruct'

# --- Data Processing ---
POS_CAT="eb"
NEG_CAT="simple_wiki"
VAL_SIZE = 400 # total across both datasets
MAX_LEN = 1024
MIN_SENTENCE_LENGTH = 3

# --- LSTM Specific ---
LSTM_HIDDEN_DIM=128
LSTM_NUM_LAYERS=4

# --- Mamba Specific ---
MAMBA_D_MODEL=128
MAMBA_NUM_LAYERS=4
MAMBA_D_STATE=16
MAMBA_DROPOUT=0.1

## Calculated Config

In [ ]:
def setup():

  training_args = SimpleNamespace(
      data_dir=DATA_DIR,
      save_dir=SAVE_DIR,
      hf_token=HF_TOKEN,
      ckpt=CKPT,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      lr=LR,
      seed=SEED,
      num_workers=NUM_WORKERS,
      pos_cat=POS_CAT,
      neg_cat=NEG_CAT,
      print_freq=PRINT_FREQ,
      model_type=MODEL_TYPE,
      lstm_hidden_dim=LSTM_HIDDEN_DIM,
      lstm_num_layers=LSTM_NUM_LAYERS,
      mamba_d_model=MAMBA_D_MODEL,
      mamba_num_layers=MAMBA_NUM_LAYERS,
      mamba_d_state=MAMBA_D_STATE,
      mamba_dropout=MAMBA_DROPOUT,
      on_colab=False,
      val_size=VAL_SIZE,
      max_len=MAX_LEN,
      min_sentence_length=MIN_SENTENCE_LENGTH,
      tokenizer_name=TOKENIZER_NAME
  )

  if 'google.colab' in sys.modules:
    training_args.on_colab = True
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    training_args.hf_token = userdata.get('HF_TOKEN')
    training_args.save_dir = "/content/drive/My Drive/"

  return training_args

## util.py

In [ ]:
def save_checkpoint(state, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(state, save_path)
    print(f"Checkpoint saved to {save_path}")

In [ ]:
def num_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def pad_mask(lengths: torch.LongTensor) -> torch.BoolTensor:
    """
    Create a mask of batch x seq where 1 is for non-padding
    and 0 is for padding.
    """
    device = lengths.device
    max_len = lengths.max().item() # .item() converts Tensor to int

    # 1. Create range [0, 1, 2, ... max_len-1]
    # Shape: (1, max_len)
    ids = torch.arange(max_len, device=device).unsqueeze(0)

    # 2. Compare with lengths
    # Shape: (batch_size, 1)
    # Broadcasting: (1, max_len) < (batch_size, 1) -> (batch_size, max_len)
    mask = ids < lengths.unsqueeze(1)

    return mask

In [ ]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries.append(time.ctime(time.time()))
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [ ]:
class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.count > 0:
            self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

## data.py

In [ ]:
def get_data_from_github(split, category):
    """
    Downloads a specific data file from the GitHub repo,
    skipping if the file already exists.
    """

    # 1. Define the URL and the local output path
    base_url = "https://raw.githubusercontent.com/latoohey/modular-fudge/refs/heads/main/data/splits/"
    filename = f"{category}.txt"
    url = f"{base_url}{split}/{category}/{filename}"

    out_dir = Path("splits") / split
    out_file_path = out_dir / filename

    # --- ADDED CHECK ---
    # 2. Check if the file already exists before doing anything else
    if out_file_path.is_file():
        print(f"File {out_file_path} already exists. Skipping download.")
        return  # Exit the function early
    # -------------------

    try:
        # 3. Create the directory (if it doesn't exist)
        out_dir.mkdir(parents=True, exist_ok=True)

        # 4. Make the HTTP request
        print(f"Downloading {url}...")
        response = requests.get(url)

        # 5. Check if the request was successful
        response.raise_for_status()

        # 6. Write the content to the file
        out_file_path.write_bytes(response.content)

        print(f"Successfully saved to {out_file_path}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
    except IOError as e:
        print(f"Error writing file: {e}")


In [ ]:
def collate(batch):
    """
    Updated collate function that pads with the correct pad_id
    instead of hardcoding 0.
    """
    # Retrieve the dynamic pad_id passed from the SplitLoader (Index 2)
    pad_id = batch[0][2]

    inputs = [b[0] for b in batch]
    lengths = torch.LongTensor([b[1] for b in batch])
    max_length = lengths.max()

    for i in range(len(inputs)):
        diff = max_length - len(inputs[i])

        if diff > 0:
            # --- FIX ---
            # Use torch.full to create a tensor filled with the specific pad_id
            padding = torch.full((diff,), pad_id, dtype=torch.long)
            inputs[i] = torch.cat([inputs[i], padding], dim=0)

    inputs = torch.stack(inputs, dim=0)

    # Get the single integer label (index 3)
    classification_labels = [b[3] for b in batch]
    classification_labels = torch.LongTensor(classification_labels)

    return (inputs, lengths, classification_labels)

In [ ]:
class Dataset:
    def __init__(self, args):
        print('Loading data...')
        self.data_dir = args.data_dir
        self.max_len = getattr(args, 'max_len', 512)

        # 1. Tokenizer Setup
        tokenizer_name = getattr(args, 'tokenizer_name', 'distilbert-base-uncased')
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

        # 2. Pad Token Logic (Restored exactly)
        if self.tokenizer.pad_token is None:
            if '<|finetune_right_pad_id|>' in self.tokenizer.get_vocab():
                self.tokenizer.pad_token = '<|finetune_right_pad_id|>'
            elif '<|reserved_special_token_0|>' in self.tokenizer.get_vocab():
                self.tokenizer.pad_token = '<|reserved_special_token_0|>'
            else:
                print("Warning: No dedicated pad token found. Using EOS token as PAD.")
                self.tokenizer.pad_token = self.tokenizer.eos_token

        self.tokenizer_pad_id = self.tokenizer.pad_token_id
        print(f"Dataset initialized. Pad Token: {self.tokenizer.pad_token} (ID: {self.tokenizer_pad_id})")

        # 3. Data Loading
        train, val, test = [], [], []

        # Helper to determine split size
        # Tries to find args.val_size, falls back to global VAL_SIZE, or 0
        target_val_size = getattr(args, 'val_size', globals().get('VAL_SIZE', 0))

        # --- LOAD TRAIN & VAL ---
        for category, label in [(args.pos_cat, 1), (args.neg_cat, 0)]:

            # A. Path Logic (Restored)
            if getattr(args, 'on_colab', False):
                file_path = os.path.join('splits', 'train', f'{category}.txt')
            else:
                file_path = os.path.join(self.data_dir, 'splits', 'train', f'{category}.txt')

            # B. Download Logic (Restored)
            if not os.path.exists(file_path):
                if getattr(args, 'on_colab', False):
                    print(f"Downloading train/{category} from GitHub...")
                    # Assumes get_data_from_github is defined in your main script context
                    get_data_from_github('train', category)
                else:
                    print(f"Warning: Train file not found at {file_path}. Skipping.")
                    continue

            # C. Processing
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as rf:
                    # We load all lines first to handle the split correctly
                    # We also apply the MIN_LENGTH filter here to keep Map-Style efficient
                    lines = [line.strip() for line in rf if len(line.strip().split()) >= args.min_sentence_length]

                    # Split Logic: First (val_size // 2) items go to Val
                    # If val_size is 0/None, we default to 10% split
                    limit = target_val_size // 2 if target_val_size > 0 else len(lines) // 10

                    val.extend([(l, label) for l in lines[:limit]])
                    train.extend([(l, label) for l in lines[limit:]])

        # --- LOAD TEST ---
        for category, label in [(args.pos_cat, 1), (args.neg_cat, 0)]:

            # A. Path Logic
            if getattr(args, 'on_colab', False):
                file_path = os.path.join('splits', 'test', f'{category}.txt')
            else:
                file_path = os.path.join(self.data_dir, 'splits', 'test', f'{category}.txt')

            # B. Download Logic
            if not os.path.exists(file_path):
                if getattr(args, 'on_colab', False):
                    print(f"Downloading test/{category} from GitHub...")
                    get_data_from_github('test', category)
                else:
                    print(f"Warning: Test file not found at {file_path}. Skipping.")
                    continue

            # C. Processing
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as rf:
                    lines = [line.strip() for line in rf if len(line.strip().split()) >= args.min_sentence_length]
                    test.extend([(l, label) for l in lines])

        self.splits = {'train': train, 'val': val, 'test': test}
        print(f"Counts - Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

    def shuffle(self, split, seed=None):
        if seed is not None:
            random.seed(seed)
        random.shuffle(self.splits[split])

    def loader(self, split, batch_size, num_workers=0, mode='fudge'):
        data_source = self.splits[split]

        if mode == 'judge':
            collate_fn = self.collate_for_judge
        else:
            collate_fn = self.collate_for_fudge

        return DataLoader(
            SplitLoader(data_source, self),
            batch_size=batch_size,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=num_workers
        )

    # --- COLLATORS ---
    def collate_for_fudge(self, batch):
        input_ids_list = [item[0] for item in batch]
        labels = torch.tensor([item[1] for item in batch], dtype=torch.long)

        padded_inputs = torch.nn.utils.rnn.pad_sequence(
            input_ids_list, batch_first=True, padding_value=self.tokenizer_pad_id
        )
        lengths = torch.tensor([len(x) for x in input_ids_list], dtype=torch.long)
        return padded_inputs, lengths, labels

    def collate_for_judge(self, batch):
        input_ids_list = [item[0] for item in batch]
        labels = [item[1] for item in batch]

        padded_inputs = torch.nn.utils.rnn.pad_sequence(
            input_ids_list, batch_first=True, padding_value=self.tokenizer_pad_id
        )
        attention_mask = (padded_inputs != self.tokenizer_pad_id).long()
        return {
            'input_ids': padded_inputs,
            'attention_mask': attention_mask,
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [ ]:
class SplitLoader(TorchDataset):
    def __init__(self, data, parent):
        self.data = data
        self.parent = parent

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        raw_sentence, label = self.data[idx]

        encoded = self.parent.tokenizer(
            raw_sentence,
            truncation=True,
            max_length=self.parent.max_len,
            add_special_tokens=True
        )
        return torch.tensor(encoded['input_ids'], dtype=torch.long), label

## Models

In [ ]:
"""
========================================================================
Model Definition File Contract
========================================================================

This file defines a classifier model architecture that is compatible with
the project's main training (`main_train.py`) and evaluation (`evaluate.py`)
scripts.

To add a new model (e.g., "MyNewModel"), create a new file like this one
(e.g., `models\my_new_model.py`) and implement the following components:

1.  A class that inherits from `torch.nn.Module`.
2.  An `__init__` method with a specific signature.
3.  An *internal* `forward` method for the model's logic.
4.  A `get_scores_for_batch` "adapter" method for training.
5.  A `get_final_scores` "adapter" method for evaluation.

The factory in `models\__init__.py` must also be updated to import
and select this new class based on the `--model_type` argument.

------------------------------------------------------------------------
CONTRACT DETAILS
------------------------------------------------------------------------

--- [1. `__init__` Method] ---

The `__init__` method *must* have the following signature:

def __init__(self, args, vocab_size, pad_id):
    ...

    - `args`: The fully populated `ArgumentParser` namespace. This
      object will contain all command-line arguments, allowing the
      model to pull its own specific hyperparameters (e.g.,
      `args.my_model_hidden_dim`, `args.my_model_num_layers`).

    - `vocab_size`: An integer (e.g., from `len(tokenizer)`)
      specifying the total vocabulary size.

    - `pad_id`: The integer index of the padding token.
      The model MUST use this to set `padding_idx` in the
      `nn.Embedding` layer. This ensures the embedding for
      padding is fixed to zero, which is critical for
      proper loss calculation and gradient safety.

--- [2. `forward` Method] ---

The `forward` method is *internal* to your model. Its signature can
be whatever you need.

    - Example: `def forward(self, inputs, lengths):` (for LSTM)
    - Example: `def forward(self, inputs):` (for Mamba/Transformer)

This method will contain the core architectural logic (embeddings,
RNN/Mamba/Transformer layers, output head).

It *must* be causal (unidirectional) and output a tensor of
per-token logits.

    - **Output Shape:** `(batch_size, seq_len)`

    - **Note on Padding:** The model is NOT required to mask/zero-out
      logits at padding positions. The training loop (`main_train.py`)
      applies a strict mask to the loss function based on sequence
      lengths. Therefore, it is acceptable for the model to output
      noise at padding indices (common in unmasked Mamba/RNNs),
      provided the causal history of *valid* tokens remains intact.

--- [3. `get_scores_for_batch` Method] ---

This is the adapter method called by `main_train.py`. It is
responsible for unpacking the batch, calling its own `forward`
method, and returning *all* per-token scores for the loss
calculation.

    - **Input:** `batch` (The raw, collated batch from the DataLoader.
      Typically `[inputs, lengths, classification_targets]`)

    - **Returns:** A tuple of `(scores, targets)`
        - `scores`: `torch.Tensor` of shape `(batch_size, seq_len)`
          (The per-token logits from the `forward` pass).
        - `targets`: `torch.Tensor` of shape `(batch_size,)`
          (The true class labels, e.g., [0, 1, 1, 0]).
    *** IMPORTANT ***
    Both `scores` and `targets` MUST be returned on the same
    DEVICE (GPU/CPU) as the model. The training loop assumes
    this adapter handles all device movement.

--- [4. `get_final_scores` Method] ---

This is the adapter method called by `evaluate.py`. It is
responsible for unpacking the batch, calling `forward`, and
returning the logit from *only* the single, final, unpadded token.

    - **Input:** `batch` (The raw, collated batch, same as above).

    - **Returns:** `last_logits`
        - `last_logits`: `torch.Tensor` of shape `(batch_size,)`
          (The logit from the last *real* token for each item
          in the batch).
"""
def comment_only():
    return

### LSTM Model

In [ ]:
# --- Model Architecture ---
class LSTMClassifier(nn.Module):

    def __init__(self, args, vocab_size, pad_id):
        """
        Initializes the LSTM model.

        Args:
            args: The full ArgumentParser namespace. Reads
                  `args.lstm_hidden_dim` and `args.lstm_num_layers`.
            vocab_size: The total vocabulary size for the embedding layer.
        """
        super().__init__()

        # --- CRITICAL CHANGE ---
        # Using `vocab_size` (e.g., 32000) is robust and correct.
        # Using `tokenizer_pad_id + 1` was brittle and would fail
        # with many tokenizers where the pad ID is not the highest ID.
        self.embed = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=args.lstm_hidden_dim,
            padding_idx=pad_id  # Use the REAL pad ID
        )
        # --- End of Change ---

        self.rnn = nn.LSTM(
            args.lstm_hidden_dim,
            args.lstm_hidden_dim,
            num_layers=args.lstm_num_layers,
            bidirectional=False,
            dropout=0.5,
            batch_first=True # Makes the permute/transpose logic simpler
        )
        self.out_linear = nn.Linear(args.lstm_hidden_dim, 1)

    def forward(self, inputs, lengths):
        """
        Internal forward pass for the LSTM.
        Requires `lengths` for sequence packing.
        """
        # (batch_size, seq_len, hidden_dim)
        embedded_inputs = self.embed(inputs)

        # Pack sequence for efficient RNN processing
        packed_inputs = pack_padded_sequence(
            embedded_inputs,
            lengths.cpu(), # Must be on CPU
            batch_first=True,
            enforce_sorted=False
        )

        # rnn_output is (packed_batch, hidden_dim)
        rnn_output, _ = self.rnn(packed_inputs)

        # Unpack: (batch_size, seq_len, hidden_dim)
        rnn_output, _ = pad_packed_sequence(
            rnn_output,
            batch_first=True
        )

        # (batch_size, seq_len)
        return self.out_linear(rnn_output).squeeze(2)

    # ---
    # --- Adapter Methods (The "Contract") ---
    # ---

    def get_scores_for_batch(self, batch):
        """
        Adapter for training.
        Unpacks batch, calls `self.forward`, and returns all scores.
        """
        # Unpack the batch as needed *by this model*
        inputs, lengths, classification_targets = batch

        # Move tensors to the model's device
        inputs = inputs.to(self.embed.weight.device)
        lengths = lengths.to(self.embed.weight.device)
        classification_targets = classification_targets.to(self.embed.weight.device)

        # Call this model's specific forward pass
        scores = self.forward(inputs, lengths)

        # Return what the training loop needs
        return scores, classification_targets

    def get_final_scores(self, batch):
        """
        Adapter for evaluation.
        Unpacks batch, calls `self.forward`, and returns final logit.
        """
        # We need all 3 components from the batch
        inputs, lengths, _ = batch

        # Move tensors to the model's device
        inputs = inputs.to(self.embed.weight.device)
        lengths = lengths.to(self.embed.weight.device)

        # Call this model's specific forward pass
        # scores shape: (batch_size, seq_len)
        scores = self.forward(inputs, lengths)

        # Find the index of the last token
        # Shape: (batch_size,)
        last_indices = (lengths - 1).long()

        # Gather the specific scores from those last indices
        # Shape: (batch_size, 1) -> (batch_size,)
        last_logits = scores.gather(
            1, last_indices.unsqueeze(1)
        ).squeeze(1)

        return last_logits

### Mamba Model

In [ ]:
"""
========================================================================
Mamba Classifier Model Definition
========================================================================

This file implements a Mamba-based classifier that follows the same
contract as the LSTM classifier for compatibility with the project's
main training (`main_train.py`) and evaluation (`evaluate.py`) scripts.

The Mamba architecture uses selective state space models (SSMs) for
efficient sequence modeling with linear complexity in sequence length.

Requirements:
- mamba-ssm (install with: pip install mamba-ssm)
- torch
- einops
"""

class MambaClassifier(nn.Module):

    def __init__(self, args, vocab_size, pad_id):
        """
        Initializes the Mamba model.

        Args:
            args: The full ArgumentParser namespace. Reads:
                  - `args.mamba_d_model` (hidden dimension, default 256)
                  - `args.mamba_d_state` (SSM state dimension, default 16)
                  - `args.mamba_d_conv` (local convolution width, default 4)
                  - `args.mamba_expand` (expansion factor, default 2)
                  - `args.mamba_num_layers` (number of Mamba blocks, default 4)
                  - `args.mamba_dropout` (dropout rate, default 0.1)
            vocab_size: The total vocabulary size for the embedding layer.
        """
        super().__init__()

        # Get hyperparameters from args with defaults
        self.d_model = getattr(args, 'mamba_d_model', 256)
        self.d_state = getattr(args, 'mamba_d_state', 16)
        self.d_conv = getattr(args, 'mamba_d_conv', 4)
        self.expand = getattr(args, 'mamba_expand', 2)
        self.num_layers = getattr(args, 'mamba_num_layers', 4)
        self.dropout_rate = getattr(args, 'mamba_dropout', 0.1)

        # Embedding layer
        self.embed = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=self.d_model,
            padding_idx=pad_id  # Use the REAL pad ID
        )

        # Dropout for regularization
        self.dropout = nn.Dropout(self.dropout_rate)

        # Stack of Mamba blocks
        self.mamba_blocks = nn.ModuleList([
            Mamba(
                d_model=self.d_model,    # Model dimension
                d_state=self.d_state,    # SSM state expansion factor
                d_conv=self.d_conv,      # Local convolution width
                expand=self.expand,      # Block expansion factor
            )
            for _ in range(self.num_layers)
        ])

        # Layer normalization between blocks
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(self.d_model)
            for _ in range(self.num_layers)
        ])

        # Final layer norm before output
        self.final_norm = nn.LayerNorm(self.d_model)

        # Output projection to single logit per token
        self.out_linear = nn.Linear(self.d_model, 1)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        """Initialize weights with sensible defaults."""
        # Initialize embeddings
        nn.init.normal_(self.embed.weight, mean=0.0, std=self.d_model ** -0.5)
        if self.embed.padding_idx is not None:
            nn.init.constant_(self.embed.weight[self.embed.padding_idx], 0)

        # Initialize output projection
        nn.init.xavier_uniform_(self.out_linear.weight)
        nn.init.constant_(self.out_linear.bias, 0)

    def forward(self, inputs, lengths=None):
        """
        Optimized Mamba forward pass.
        We do NOT mask internally. We allow Mamba to process padding tokens
        naturally to preserve state dynamics. The training loop handles
        ignoring the resulting padding outputs.
        """
        # Embed tokens: (batch_size, seq_len, d_model)
        x = self.embed(inputs)
        x = self.dropout(x)

        # CRITICAL: Mamba kernels require contiguous memory layout
        x = x.contiguous()

        # Process through Mamba blocks
        for mamba_block, layer_norm in zip(self.mamba_blocks, self.layer_norms):
            # Pre-norm architecture
            residual = x
            x = layer_norm(x)

            # Block
            x = mamba_block(x)

            # Residual connection
            # NOTE: We removed "if mask is not None: x = x * mask"
            # to allow correct State Space Model evolution.
            x = residual + self.dropout(x)

        # Final normalization
        x = self.final_norm(x)

        # Project to logits: (batch_size, seq_len)
        scores = self.out_linear(x).squeeze(-1)

        return scores

    # ---
    # --- Adapter Methods (The "Contract") ---
    # ---

    def get_scores_for_batch(self, batch):
        """
        Adapter for training.
        Unpacks batch, calls `self.forward`, and returns all scores.

        Args:
            batch: The raw, collated batch from the DataLoader.
                   Typically [inputs, lengths, classification_targets]

        Returns:
            scores: torch.Tensor of shape (batch_size, seq_len)
            targets: torch.Tensor of shape (batch_size,)
        """
        # Unpack the batch
        inputs, lengths, classification_targets = batch

        # Move tensors to the model's device
        inputs = inputs.to(self.embed.weight.device)
        lengths = lengths.to(self.embed.weight.device)
        classification_targets = classification_targets.to(self.embed.weight.device)

        # Call forward pass
        scores = self.forward(inputs, lengths)

        # Return scores and targets
        return scores, classification_targets

    def get_final_scores(self, batch):
        """
        Adapter for evaluation.
        Unpacks batch, calls `self.forward`, and returns final logit.

        Args:
            batch: The raw, collated batch from the DataLoader.

        Returns:
            last_logits: torch.Tensor of shape (batch_size,)
                        The logit from the last real token for each item.
        """
        # Unpack the batch
        inputs, lengths, _ = batch

        # Move tensors to the model's device
        inputs = inputs.to(self.embed.weight.device)
        lengths = lengths.to(self.embed.weight.device)

        # Call forward pass
        # scores shape: (batch_size, seq_len)
        scores = self.forward(inputs, lengths)

        # Find the index of the last token for each sequence
        # Shape: (batch_size,)
        last_indices = (lengths - 1).long()

        # Gather the scores from the last valid position
        # Shape: (batch_size, 1) -> (batch_size,)
        last_logits = scores.gather(
            1, last_indices.unsqueeze(1)
        ).squeeze(1)

        return last_logits

### model init

In [ ]:
def get_model(args, vocab_size, pad_id):
    """
    This factory function reads the --model_type argument
    and returns the correct, initialized model.
    """
    if args.model_type == 'lstm':
        return LSTMClassifier(args, vocab_size, pad_id)
    elif args.model_type == 'mamba':
        return MambaClassifier(args, vocab_size, pad_id)
    else:
        raise ValueError(f"Unknown model type: {args.model_type}")

## main_train.py

In [ ]:
def train(model, dataset, optimizer, criterion, epoch, args):
    model.train()
    dataset.shuffle('train', seed=epoch + args.seed)

    loader = dataset.loader('train', args.batch_size, num_workers=args.num_workers)
    loss_meter = AverageMeter('loss', ':6.4f')
    total_length = len(loader)
    progress = ProgressMeter(total_length, [loss_meter], prefix='Training: ')

    for batch_num, batch in enumerate(tqdm(loader, total=len(loader))):
        # 1. Use the Contract to get scores and targets
        # (The model handles device movement and internal unpacking)
        scores, classification_targets = model.get_scores_for_batch(batch)

        # 2. We still need 'lengths' for the mask logic below
        _, lengths, _ = batch
        lengths = lengths.to(scores.device) # Ensure it matches the GPU/CPU of scores

        # --- This is the "Implicit Prefix" Loss Logic ---

        expanded_labels = classification_targets.unsqueeze(1).expand(-1, scores.shape[1])

        # 2. Get padding mask (batch_size, seq_len)
        length_mask = pad_mask(lengths) # 1 for real tokens, 0 for padding

        # 3. Flatten scores, labels, and mask
        scores_flat = scores.flatten()
        labels_flat = expanded_labels.flatten().float()
        mask_flat = length_mask.flatten()

        # 4. Select only the non-padded tokens for loss calculation
        scores_unpadded = scores_flat[mask_flat == 1]
        labels_unpadded = labels_flat[mask_flat == 1]

        # 5. Calculate loss
        loss = criterion(scores_unpadded, labels_unpadded)
        # --- End of Implicit Logic ---

        optimizer.zero_grad()
        loss.backward()

        # --- SANITY CHECK ---
        if dataset.tokenizer_pad_id is not None:
            # Assuming the model has an 'embed' layer.
            # If you change model types (e.g. to Mamba), ensure this attribute path exists.
            pad_grad = model.embed.weight.grad[dataset.tokenizer_pad_id]

            # It's possible pad_grad is None if the batch had NO padding (unlikely but possible)
            if pad_grad is not None:
                if torch.sum(torch.abs(pad_grad)) > 0:
                    raise RuntimeError(f"FATAL: Model is learning from PAD token {dataset.tokenizer_pad_id}!")
        # --------------------

        optimizer.step()

        loss_meter.update(loss.item(), scores.size(0))
        if batch_num % args.print_freq == 0:
            progress.display(batch_num)

    progress.display(total_length)

In [ ]:
def validate(model, dataset, criterion, args):
    model.eval()
    loader = dataset.loader('val', args.batch_size, num_workers=args.num_workers)

    loss_meter = AverageMeter('loss', ':6.4f')
    acc_meter = AverageMeter('acc', ':6.2f') # Add an accuracy meter

    total_length = len(loader)
    # Add acc_meter to the display list
    progress = ProgressMeter(total_length, [loss_meter, acc_meter], prefix='Validation: ')

    with torch.no_grad():
        for batch_num, batch in enumerate(tqdm(loader, total=len(loader))):

            # 1. Use the Contract (Handles device movement & unpacking)
            scores, classification_targets = model.get_scores_for_batch(batch)

            # 2. Get lengths specifically for masking (Manual step)
            _, lengths, _ = batch
            lengths = lengths.to(scores.device)

            # --- Identical Implicit Loss Logic ---
            expanded_labels = classification_targets.unsqueeze(1).expand(-1, scores.shape[1])
            length_mask = pad_mask(lengths)

            scores_flat = scores.flatten()
            labels_flat = expanded_labels.flatten().float()
            mask_flat = length_mask.flatten()

            scores_unpadded = scores_flat[mask_flat == 1]
            labels_unpadded = labels_flat[mask_flat == 1]

            if scores_unpadded.nelement() > 0:
                # Calculate Loss
                loss = criterion(scores_unpadded, labels_unpadded)

                # --- Calculate Accuracy ---
                # Sigmoid converts logits to probs (0 to 1)
                probs = torch.sigmoid(scores_unpadded)
                # Anything > 0.5 is a prediction for Class 1 (EB11)
                preds = (probs > 0.5).float()
                # Compare preds to true labels
                acc = (preds == labels_unpadded).float().mean()
                # --------------------------

                # Update meters (Use scores.size(0) for batch size)
                loss_meter.update(loss.item(), scores.size(0))
                acc_meter.update(acc.item(), scores.size(0))

    progress.display(total_length)

    # Return loss (or accuracy, depending on what you want to track for early stopping)
    return loss_meter.avg

In [ ]:
def dry_run(args, model, dataset):
    print("--- INITIATING DRY RUN ---")

    # Ensure model is in training mode so gradients are generated
    model.train()

    # 1. Get one batch
    loader = dataset.loader('train', batch_size=2)
    batch = next(iter(loader))

    # 2. Forward Pass
    # Your model's 'get_scores_for_batch' handles moving tensors to GPU,
    # so we don't need to manually .to(device) the batch here.
    scores, targets = model.get_scores_for_batch(batch)

    # 3. Backward Pass (Dummy Loss)
    # We just need a scalar to test that backprop works without OOMing.
    # We don't need the real loss function here.
    loss = scores.sum()
    loss.backward()

    # 4. Check Max ID vs Vocab Size (The critical check for your bug)
    input_ids = batch[0]
    # Access the embedding weight size directly
    vocab_limit = model.embed.weight.size(0)
    if input_ids.max() >= vocab_limit:
        raise ValueError(f"CRITICAL: Batch contains Token ID {input_ids.max()}, but Embedding size is only {vocab_limit}. Adjust get_model()!")

    # 5. Save/Load Check (Verifies serialization)
    torch.save(model.state_dict(), "dry_run.pth")
    model.load_state_dict(torch.load("dry_run.pth"))

    print("--- DRY RUN PASSED ---")

    # Clean up
    if os.path.exists("dry_run.pth"):
        os.remove("dry_run.pth")

    # Zero gradients so this test doesn't affect the first real training step
    model.zero_grad()


In [ ]:
def evaluate_test_set(model, dataset, args, checkpoint_path):
    print(f"\n--- STARTING FINAL EVALUATION ---")
    print(f"Loading best checkpoint from: {checkpoint_path}")

    # Load the best weights
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model.load_state_dict(checkpoint['state_dict'])

    model.eval()
    loader = dataset.loader('test', args.batch_size, num_workers=args.num_workers)

    all_preds = []
    all_targets = []

    print("Running inference on Test Set...")
    with torch.no_grad():
        for batch in tqdm(loader, total=len(loader)):
            # 1. Get the logits for the LAST token in the sequence
            # We use get_final_scores because we care about the document-level decision,
            # not the 'loss at every step' intermediate predictions.
            logits = model.get_final_scores(batch)

            # 2. Convert logits to probabilities and then to binary predictions
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).long()

            # 3. Get actual labels (Index 2 in your collate tuple)
            # (inputs, lengths, classification_labels)
            targets = batch[2]

            # 4. Move to CPU and collect
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    # --- Calculate Metrics ---
    acc = accuracy_score(all_targets, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='binary')
    conf_mat = confusion_matrix(all_targets, all_preds)

    print("\n" + "="*30)
    print("TEST SET RESULTS")
    print("="*30)
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("-" * 30)
    print("Confusion Matrix:")
    print(conf_mat)
    print("-" * 30)

    # Detailed report useful for seeing class imbalances
    print("Classification Report:")
    print(classification_report(all_targets, all_preds, target_names=['Negative', 'Positive']))

    return f1

In [ ]:
def seed_everything(seed=42):
    # 1. Set the python built-in random seed
    random.seed(seed)

    # 2. Set the numpy seed
    np.random.seed(seed)

    # 3. Set the pytorch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # If using multi-GPU

    # 4. Important: Force CuDNN to be deterministic
    # This slows down training slightly but ensures 'exact' reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 5. Set hashing seed (vital for dictionary ordering/hashing)
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Global seed set to {seed}")

In [ ]:
def main():

    seed_everything(SEED)

    args = setup()
    login(token=args.hf_token)

    # Hard-code the task
    args.task = 'transfer'
    args.device = torch.device(DEVICE)

    dataset = Dataset(args)
    os.makedirs(args.save_dir, exist_ok=True)

    model = get_model(args, len(dataset.tokenizer), dataset.tokenizer_pad_id)
    model = model.to(args.device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

    # --- ADDED: Scheduler ---
    # If validation loss doesn't improve for 2 epochs, cut LR by half
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
      optimizer, mode='min', factor=0.5, patience=2
    )

    best_val_metric = 1e8

    print('Model Parameters:', num_params(model))
    criterion = nn.BCEWithLogitsLoss().to(args.device)

    dry_run(args, model, dataset)

    now = datetime.now()
    date_string = now.strftime("%Y%m%d_%H%M%S")

    save_path = os.path.join(args.save_dir, f'{args.model_type}_{date_string}.pth.tar')

    # --- ADDED: Early Stopping Config ---
    early_stopping_patience = 5
    epochs_no_improve = 0
    # ------------------------------------

    for epoch in range(args.epochs):
        print(f"--- TRAINING: Epoch {epoch} at {time.ctime()} ---")
        train(model, dataset, optimizer, criterion, epoch, args)

        print(f"--- VALIDATION: Epoch {epoch} at {time.ctime()} ---")
        metric = validate(model, dataset, criterion, args)

        # Update the scheduler based on validation loss
        scheduler.step(metric)

        # --- ADD THIS TO REPLACE VERBOSE=TRUE ---
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Current Learning Rate: {current_lr}")
        # ----------------------------------------

        if metric < best_val_metric:
            print(f'New best val metric: {metric:.4f}')
            best_val_metric = metric

            # Reset patience because we improved
            epochs_no_improve = 0

            save_checkpoint({
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'best_metric': best_val_metric,
                'optimizer': optimizer.state_dict(),
                'args': args
            }, save_path)
        else:
            # We didn't improve
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epochs.")

            if epochs_no_improve >= early_stopping_patience:
                print(f"Early stopping triggered after epoch {epoch}!")
                print(f'Final val metric: {best_val_metric:.4f}')
                break
    evaluate_test_set(model, dataset, args, save_path)

In [ ]:
main()